## 주식 가격 (대성에너지)


### 저장된 데이터 마지막 일자의 다음날부터 현재일 기준 직전일까지 데이터를 일괄로 가져와서 저장

In [1]:
stock_code = '117580'   # 대성에너지 stock code

In [2]:
import sys
module_path = "D:\PythonProject\data-gatherer\common"
sys.path.append(module_path)
import dbconnect
import myutil

#### 0. 데이터를 가져올 날짜 설정

In [3]:
conn = dbconnect.db_connect("DEMO_DW")
cur = conn.cursor()

query = f'SELECT tr_date FROM stock_price where stock_code = {stock_code} ORDER BY tr_date DESC LIMIT 1'
cur.execute(query)

row = cur.fetchone()
last_date = row[0]     # 저장된 데이터의 마지막 일자

conn.close()

In [4]:
start_date = myutil.get_next_day(last_date)
end_date = myutil.get_previous_day()

In [5]:
stock_code, last_date, start_date, end_date

('117580', '20230705', '20230706', '20230709')

#### 1. 주가 데이터 가져오기

In [6]:
from pykrx import stock

# 특정기간 특정 종목의 OHLCV
ohlcv = stock.get_market_ohlcv(start_date, end_date, stock_code)

#### 2. 데이터 변환 및 전처리

In [8]:
df_ohlcv = ohlcv.reset_index()
# 날짜 형식 변환 => 문자열 yyyymmdd
df_ohlcv['날짜'] = df_ohlcv['날짜'].astype('str')
df_ohlcv['날짜'] = df_ohlcv['날짜'].str.replace('-', '')

In [ ]:
# ohlcv.shape
# df_ohlcv.dtypes

#### 3. DB 테이블에 저장

In [13]:
conn = dbconnect.db_connect("DEMO_DW")
cur = conn.cursor()

for row in df_ohlcv.itertuples():
    sql = "insert into stock_price (stock_code, tr_date, open, high, low, close, volume, tr_amount, fluctuation_rate) \
                            values (%s, %s, %s, %s, %s, %s, %s, %s, %s)"
    cur.execute(sql, (stock_code, row[1], row[2], row[3], row[4], row[5], row[6], row[7], round(row[8],2)))
conn.commit()
conn.close()